# Logistic Regression

In this notebook, we implement three logistic regression classifiers with different features. All models are using $L_1$ penalty, and the regularization power is tuned using the Nested Cross-Validation scheme.

## 1. Nested Cross-Validation Scheme

For all models having hyper-parameters, we use the same Nested Cross-Validation method to tune, train and test models. We iterate test donor from donor 1, 2, 3, 5 and 6. Each test donor is separated from the tuning and training process of its associated model. During tuning, the validation set is iterated through the left 4 donors. The average of performances on 4 validation sets is used to determine the best parameter for one test donor. Therefore, each test donor might have different optimal parameters.

<img src="./plots/cross_validation.pdf" width="600">

## 2. Logistic Regression with Image Pixel

### 2.1. Data Preparation

For the first logistic regression model, we use the $78\times 78$ pixel matrix as the representation of each image.

In [70]:
import numpy as np
import cv2
import re
import pandas as pd
from glob import glob
from os.path import basename, join, exists
from json import load, dump
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from IPython.display import display, Markdown

In [59]:
all_data = {}

# Load the pixel matrix for each donor
for d in [1, 2, 3, 5, 6]:
    no_aug_x, no_aug_y = [], []
    aug_x, aug_y = [], []
    
    for name in glob("./images/sample_images/processed/augmented/donor_{}/*/*.png".format(d)):
        pixel_mat = cv2.imread(name, 0)
        pixel_feature = pixel_mat.reshape((1, -1))
        
        # Add the feature and label to the correct list
        base_name = basename(name)
        cur_label = 0 if 'noact' in base_name else 1
        
        if 'r' not in base_name and 'f' not in base_name:
            no_aug_x.append(pixel_feature)
            no_aug_y.append(cur_label)
        
        aug_x.append(pixel_feature)
        aug_y.append(cur_label)

    assert(len(aug_x) == len(aug_y))
    assert(len(no_aug_x) == len(no_aug_x))
    
    # Add data for this donor
    all_data[d] = {
        'no_aug_x': np.vstack(no_aug_x),
        'no_aug_y': np.array(no_aug_y),
        'aug_x': np.vstack(aug_x),
        'aug_y': np.array(aug_y)
    }

In [60]:
all_data[1]

{'no_aug_x': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'no_aug_y': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0]),
 'aug_x': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 

### 2.2. Model Tuning/Training

After setting up the training data, we can use Nested Cross-Validation to tune the regularization parameter for each test donor. We use grid search to try all $\lambda$ candidates $[0.0001, 0.001, 0.01, 0.1, 1]$. We use average precision to choose the best parameter.

In [61]:
def nested_cv(all_data, c_candidates):
    """
    Use nested cross validation to find the best parameter
    for each test donor.
    
    Args:
        all_data(dict): mapping each donor to four lists: augmented features,
            augmented labels, un-augmented features, un-augmented labels
        c_candidates(list): the candidate for lambda value
        
    Return:
        dict: mapping each donor to its best lambda value
    """

    best_parameters = {}
    for tid in [1, 2, 3, 5, 6]:

        # Validate 4 times for each parameter
        vids = [i for i in [1, 2, 3, 5, 6] if i != tid]

        # This array mirrors the same index of c_candidates
        average_aps = np.zeros(len(c_candidates))

        for i in range(len(c_candidates)):
            c = c_candidates[i]
            lr_model = LogisticRegression(penalty='l1', C=c, class_weight='balanced')
            aps = []

            for vid in vids:
                # Training set includes augmented images from {all donor - test donor - vali donor}
                train_x = np.vstack([all_data[i]['aug_x'] for i in [1, 2, 3, 5, 6]
                                     if i != tid and i != vid])
                train_y = np.hstack([all_data[i]['aug_y'] for i in [1, 2, 3, 5, 6]
                                     if i != tid and i != vid])
                assert(train_x.shape[0] == train_y.shape[0])

                # Validation set includes non-augmented images from {vali donor}
                vali_x = all_data[vid]['aug_x']
                vali_y = all_data[vid]['aug_y']

                # Train the model on the trining set
                lr_model.fit(train_x, train_y)

                # Test the model on the validation set
                predicted_score = lr_model.predict_proba(vali_x)

                # Many metric functions in sklearn requires 1d score array (pos proba)
                predicted_score_1d = [s[1] for s in predicted_score]

                # Compute the average precision for the current validation
                aps.append(metrics.average_precision_score(vali_y, predicted_score_1d))

            # Compute the average of 4 validation runs
            average_aps[i] = np.mean(aps)
            print("The average performance of parameter {} for test donor {} is {:.4f}.".format(
                c, tid, np.mean(aps)
            ))

        # Now, we have exhaustively considered all parameter candidates. Choose the one giving the
        # best mean average precision score
        current_best_para = c_candidates[np.argmax(average_aps)]
        print("The best parameter for test donor {} is {}".format(tid, current_best_para))
        best_parameters[tid] = current_best_para
    
    return best_parameters

In [62]:
# Smaller c gives stronger regularization power
c_candidates = [0.0001, 0.001, 0.01, 0.1, 1]
best_parameters = nested_cv(all_data, c_candidates)

The average performance of parameter 0.0001 for test donor 1 is 0.8697.
The average performance of parameter 0.001 for test donor 1 is 0.8773.
The average performance of parameter 0.01 for test donor 1 is 0.8254.
The average performance of parameter 0.1 for test donor 1 is 0.8039.
The average performance of parameter 1 for test donor 1 is 0.7587.
The best parameter for test donor 1 is 0.001
The average performance of parameter 0.0001 for test donor 2 is 0.7590.
The average performance of parameter 0.001 for test donor 2 is 0.7807.
The average performance of parameter 0.01 for test donor 2 is 0.7191.
The average performance of parameter 0.1 for test donor 2 is 0.6788.
The average performance of parameter 1 for test donor 2 is 0.6137.
The best parameter for test donor 2 is 0.001
The average performance of parameter 0.0001 for test donor 3 is 0.7685.
The average performance of parameter 0.001 for test donor 3 is 0.8133.
The average performance of parameter 0.01 for test donor 3 is 0.7455.

In [63]:
best_parameters

{1: 0.001, 2: 0.001, 3: 0.001, 5: 0.001, 6: 0.001}

After tuning the model, we found $0.001$ is the best $\lambda$ for all 5 test donors.

### 2.3. Model Testing

Then, we can use the best parameter to train a new model on all augmented images in the new training set, and test it on un-augmented images from each test donor.

In [56]:
def get_score(model, x_test, y_test, pos=1):
    """
    This funciton runs the trained `model` on `x_test`, compares the test
    result with `y_test`. Finally, it outputs a collection of various
    classificaiton performance metrics.
    
    Args:
        model: a trained sklearn model
        x_test(np.array(m, n)): 2D feature array in the testset, m elements and each
            element has n features
        y_test(np.array(m)): 1D label array in the testset. There are m entries.
        pos: the encoding of postive label in `y_test`

    Return:
        A dictionary containing the metrics information and predictions:
            metrics scores: ['acc': accuracy, 'precision', 'recall', 'ap': average precision,
                             'aroc': area under ROC curve, 'pr': PR curve points,
                             'roc': ROC curve points]
            predicitons: ['y_true': the groundtruth labels, 'y_score': predicted probability]
    """

    y_predict_prob = model.predict_proba(x_test)
    y_predict = model.predict(x_test)

    # Sklearn requires the prob list to be 1D
    y_predict_prob = [x[pos] for x in y_predict_prob]
    y_test_fixed = np.array(y_test)
    
    if pos == 0:
        # Flip the array so 1 represents the positive class
        y_test_fixed = 1 - np.array(y_test)

    # Compute the PR-curve points
    precisions, recalls, thresholds = metrics.precision_recall_curve(
        y_test_fixed,
        y_predict_prob,
        pos_label=pos
    )

    # Compute the roc-curve points
    fprs, tprs, roc_thresholds = metrics.roc_curve(y_test_fixed, y_predict_prob,
                                                   pos_label=pos)

    return ({'acc': metrics.accuracy_score(y_test_fixed, y_predict),
             'precision': metrics.precision_score(y_test_fixed, y_predict,
                                                  pos_label=pos),
             'recall': metrics.recall_score(y_test_fixed, y_predict,
                                            pos_label=pos),
             'ap': metrics.average_precision_score(y_test_fixed,
                                                   y_predict_prob),
             'aroc': metrics.roc_auc_score(y_test_fixed,
                                           y_predict_prob),
             'pr': [precisions.tolist(), recalls.tolist(),
                    thresholds.tolist()],
             'roc': [fprs.tolist(), tprs.tolist(), roc_thresholds.tolist()],
             'y_true': y_test,
             'y_score': y_predict_prob})

In [57]:
def make_table(metric_dict, count_dict):
    """
    Transfer the model performance metric dictionary into a Markdown table.
    
    Args:
        metric_dict(dict): a dictionary encoding model performance statisitcs
            and prediction information for all test donors
        count_dict(dict): a dinctionary encoding the count of activated and quiescent
            samples for each test donor
    
    Return:
        string: a Markdown syntax table
    """

    # Define header and line template
    table_str = ""
    line = "|{}|{:.2f}%|{:.2f}%|{:.2f}%|{:.2f}%|{:.2f}%|{}|{}|\n"
    table_str += ("|Test Donor|Accuracy|Precision|Recall|Average Precision|AUC|Num of Activated|Num of Quiescent|\n")
    table_str += "|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|\n"

    for d in [1, 2, 3, 5, 6]:
        result = metric_dict[d]
        table_str += (line.format("donor_{}".format(d),
                                  result['acc'] * 100, result['precision'] * 100,
                                  result['recall'] * 100, result['ap'] * 100,
                                  result['aroc'] * 100, count_dict[d]['activated'],
                                  count_dict[d]['quiescent']))

    return table_str

In [68]:
model_performance = {}

for tid in [1, 2, 3, 5, 6]:
    # Collect the performance metrics for each test donor tid
    lr_model = LogisticRegression(penalty='l1', C=best_parameters[d], class_weight='balanced')
    
    # Training set includes augmented images from {all donor - test donor}
    train_x = np.vstack([training_data[i]['aug_x'] for i in [1, 2, 3, 5, 6] if i != tid])
    train_y = np.hstack([training_data[i]['aug_y'] for i in [1, 2, 3, 5, 6] if i != tid])
    assert(train_x.shape[0] == train_y.shape[0])

    # Test set includes non-augmented images from {test donor}
    test_x = training_data[tid]['no_aug_x']
    test_y = training_data[tid]['no_aug_y']
    
    lr_model.fit(train_x, train_y)
    
    model_performance[tid] = get_score(lr_model, test_x, test_y)

In [69]:
# Count the lables for each donor
count_dict = {}

for d in [1, 2, 3, 5, 6]:
    act_count = len(glob("./images/sample_images/processed/augmented/donor_{}/activated/*.png".format(d)))
    qui_count = len(glob("./images/sample_images/processed/augmented/donor_{}/quiescent/*.png".format(d)))
    count_dict[d] = {
        'activated': act_count,
        'quiescent': qui_count
    }

In [71]:
# Create a table summary
display(Markdown(make_table(model_performance, count_dict)))

|Test Donor|Accuracy|Precision|Recall|Average Precision|AUC|Num of Activated|Num of Quiescent|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|donor_1|65.00%|21.92%|72.73%|46.30%|73.82%|132|948|
|donor_2|60.00%|92.31%|55.38%|92.45%|71.18%|390|90|
|donor_3|70.76%|47.62%|86.96%|78.57%|86.85%|276|750|
|donor_5|67.39%|84.91%|67.16%|90.55%|74.45%|402|150|
|donor_6|81.37%|72.73%|90.91%|89.39%|89.77%|264|348|


As we can see, the average accuracy for all test donors are above $50\%$. A simple logistic regression model with basic feature encoding can give better predictions than random guesses.

### 2.4. Model Interpretation

## 3. Logistic Regression with Image Size and Total Intensity

### 3.1. Data Preparation

### 3.2. Model Tuning/Training

### 3.3. Model Testing


## 4. Logistic Regression with CellProfiler Features


### 4.1. Data Preparation

### 4.2. Model Tuning/Training

### 4.3. Model Testing

### 4.4. Model Interpretation

## 5. Summary